In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Load Data
df = pd.read_csv('database.csv', parse_dates=['Time'])
df

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,2023-03-11 13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,2023-03-11 11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,2023-03-11 18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,2023-03-11 18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,2023-03-11 13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,2023-03-11 08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,...,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,2023-03-11 09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,...,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,2023-03-11 12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,...,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,2023-03-11 22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,...,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


In [2]:
# Filter Data
df = df[df['Magnitude'] >= 5.0]
# df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Extract Features
df['Time'] = pd.to_datetime(df['Time'],utc=True)
df['hour'] = df['Time'].dt.hour
df['day'] = df['Time'].dt.day
df['month'] = df['Time'].dt.month
df['year'] = df['Time'].dt.year
df['weekday'] = df['Time'].dt.weekday

# Create Target Variable
df['target'] = (df['Magnitude'] >= 6.0).astype(int)

# Select Features
features = ['Latitude', 'Longitude', 'Depth', 'hour', 'day', 'month', 'year', 'weekday']
X = df[features]
y = df['target']

# Standardize Data
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [3]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on Test Data
y_pred = model.predict(X_test)

# Evaluate Model
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6564168268204142
              precision    recall  f1-score   support

           0       0.70      0.86      0.77      3205
           1       0.41      0.21      0.28      1478

    accuracy                           0.66      4683
   macro avg       0.56      0.54      0.53      4683
weighted avg       0.61      0.66      0.62      4683



In [11]:
# Load Latest Earthquake Data
latest_data = pd.read_csv('database.csv', parse_dates=['Time'])

# Extract Features
latest_data['Time'] = pd.to_datetime(df['Time'],utc=True)
latest_data['hour'] = latest_data['Time'].dt.hour
latest_data['day'] = latest_data['Time'].dt.day
latest_data['month'] = latest_data['Time'].dt.month
latest_data['year'] = latest_data['Time'].dt.year
latest_data['weekday'] = latest_data['Time'].dt.weekday

# Select Features
latest_X = latest_data[features]

# Standardize Data
latest_X = scaler.transform(latest_X)

# Make Predictions
latest_pred = model.predict(latest_X)

# Print Results
if latest_pred[0] == 1:
    print('An earthquake with a magnitude of 6 or higher is likely to occur in the next 24 hours.')
else:
    print('An earthquake with a magnitude of 6 or higher is unlikely to occur in the next 24 hours.')


An earthquake with a magnitude of 6 or higher is likely to occur in the next 24 hours.


SyntaxError: invalid syntax (2830201818.py, line 1)